# Catchment masks
***


**Author:** Chus Casado Rodríguez<br>
**Date:** 30-09-2024<br>

**Introduction:**<br>
This notebook reads the upstream area maps for each catchment created with the `cutmaps` tool, converts it into masks, and export them as NetCDF files.

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
# import rioxarray
from tqdm.auto import tqdm

from reservoirs_lshm.utils import DatasetConfig

## Configuration

In [2]:
cfg = DatasetConfig('../ResOpsUS/config_dataset.yml')

print(f'Attribute tables will be saved in {cfg.PATH_ATTRS}')

Attribute tables will be saved in Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\v2.0\attributes


## Data

In [8]:
reservoirs = pd.read_csv(cfg.PATH_RESOPS / 'ancillary' / 'cutmaps' / 'points.txt', header=None, sep='\t')
reservoirs.columns = ['lat', 'lon', 'ID']
reservoirs.set_index('ID', drop=True, inplace=True)

print(f'{reservoirs.shape[0]} reservoirs in the input points')

119 reservoirs in the input points


## Create masks

In [9]:
# load catchment maps derived using `cutmaps`
path = cfg.PATH_RESOPS / 'ancillary' / 'cutmaps'
mask_map = 'upArea_repaired.nc'

masks = {}
directories = [dir for dir in path.iterdir() if dir.is_dir()]
for directory in tqdm(directories):
    try:
        # reservoir ID
        ID = int(directory.stem)
        
        # load upstream area map
        mask = xr.open_dataset(directory / mask_map)['Band1']
        mask.name = str(ID)
        
        # create and save a mask out of the previous map
        mask = xr.where(mask.notnull(), 1, mask)
        masks[ID] = mask
    except Exceptio as e:
        print(e)
        continue

  0%|          | 0/119 [00:00<?, ?it/s]

In [12]:
# export
PATH_OUT = cfg.PATH_RESOPS / 'ancillary' / 'catchstats' / 'masks'
PATH_OUT.mkdir(parents=True, exist_ok=True)1
for ID, mask in tqdm(masks.items()):
    mask.to_netcdf(PATH_OUT / f'{ID}.nc')

  0%|          | 0/119 [00:00<?, ?it/s]